## Exercice 1 : VAE sur MNIST
- Implémentez un VAE simple pour générer des chiffres MNIST.
- Visualisez l'espace latent en 2D.
- Interprétez la structure obtenue.

In [ ]:
# Importation des librairies
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

## Exercice 2 : GAN sur MNIST
- Implémentez un GAN simple pour générer des chiffres MNIST.
- Comparez les images générées avec les vraies.
- Analysez la stabilité de l'entraînement.

In [ ]:
# Structure de base GAN
class Generator(nn.Module):
    def __init__(self, z_dim=100, img_dim=784):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 128),
            nn.ReLU(),
            nn.Linear(128, img_dim),
            nn.Tanh()
        )
    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, img_dim=784):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(img_dim, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

## Exercice 3 : Flows (RealNVP, Glow)
- Explorez la structure d'un flow normalisant simple.
- Testez sur des données synthétiques.
- Visualisez la transformation de la distribution.

In [ ]:
# Exemple de transformation bijective simple
import numpy as np
import seaborn as sns

# Transformation affine
def affine_flow(x, a=2.0, b=1.0):
    return a * x + b

x = np.random.normal(0, 1, 1000)
y = affine_flow(x)
sns.histplot(y, kde=True)

## Exercice 4 : Modèle de diffusion sur MNIST
- Implémentez un modèle de diffusion simple (DDPM) pour générer des images MNIST.
- Visualisez le processus de bruitage et de génération.
- Comparez la qualité des images générées avec celles des autres modèles.

In [ ]:
# Modèle de diffusion simple (DDPM) sur MNIST
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = transforms.ToTensor()
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

# Réseau de bruitage simple (UNet simplifié)
class SimpleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 784)
        )
    def forward(self, x, t):
        return self.net(x)

# Paramètres de diffusion
T = 100
betas = torch.linspace(1e-4, 0.02, T)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, dim=0)

# Fonction de bruitage
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)
    sqrt_alphas_cumprod_t = alphas_cumprod[t].sqrt().to(device)
    sqrt_one_minus_alphas_cumprod_t = (1 - alphas_cumprod[t]).sqrt().to(device)
    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

# Entraînement du modèle de diffusion
model = SimpleUNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    for x, _ in loader:
        x = x.view(-1, 784).to(device)
        t = torch.randint(0, T, (x.size(0),), device=device)
        noise = torch.randn_like(x)
        x_noisy = q_sample(x, t, noise)
        pred_noise = model(x_noisy, t)
        loss = nn.functional.mse_loss(pred_noise, noise)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# Visualisation du bruitage progressif
x, _ = next(iter(loader))
x = x[:8].view(-1, 784).to(device)
fig, axes = plt.subplots(1, 8, figsize=(16,2))
for i in range(8):
    t = int(i * T / 8)
    x_noisy = q_sample(x, t)
    axes[i].imshow(x_noisy[0].cpu().view(28,28), cmap='gray')
    axes[i].set_title(f't={t}')
    axes[i].axis('off')
plt.suptitle('Processus de bruitage (MNIST, Diffusion)')
plt.show()